<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/PuLP%20-%20Tutorial/Tutorial%20PuLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial PuLP

This tutorial uses the well-known Traveling-Salesperson-Problem (TSP) to explain how Python's module PuLP can be used to model and solve simple Linear Programs. A Tutorial on the implementaion of the TSP in CPLEX can be found here: [TSP Tutorial on CPLEX](https://www.scm.bwl.uni-kiel.de/de/lehre/tutorial-on-cplex.pdf).



## Problem Description

The Traveling Salesperson Problem (TSP) is one of the most famous combinatorial problems in the fields of mathematics, computer science, and operations research. The classical definition of the TSP is as follows: What is the shortest possible route for a traveling salesperson seeking to visit each city on a list exactly once and return to his city of origin? (Cook, William (2012) *In Pursuit of the Travelin Salesman*).

While the problem is easy to understand and easy to formulate as a mathematical model, solving it to optimality through complete enumeration of all feasible solutions quickly becomes intractable as the number of potential solutions grows exponentially in the number of cities to visit.

To date, there exists no algorithm that solves the TSP to optimality in polynomial time and it is widely believed that there is no such algorithm. However, heuristics are usually capable of finding very good solutions in a short computation time. 

Let's continue with an artificial small-sized data set that is used throughout this tutorial. The example consist of five cities (1 to 5) in which city 1 is the salesperson's city of origin (also referred to as depot). The following Table shows the distance between the cities. 

\begin{array}{l lllll}
         destination \rightarrow \\
         \hline
         origin \downarrow & 1 & 2 & 3 & 4 & 5 \\
         \hline
         1 & - & 7 & 8 & 5 & 3 \\
         2 & 7 & - & 2 & 8 & 10 \\
         3 & 8 & 2 & - & 6 & 9 \\
         4 & 5 & 9 & 6 & - & 4 \\
         5 & 3 & 10 & 9 & 4 & - \\
\end{array}


## Mathematical Formulation


A graph-based formulation is used that is based on the paper from Langevin, André, Francois Soumis, and Jacques Desrosiers ("Classification of travelling salesman problem formulations". In: Operations Research Letters. 1990). For the subtour constraints, the popular formulation from Miller, Tucker and Zemlin is used. 

Let $N$ denote the number of nodes in the network, i.e., the number of cities. The distance for the salesperson to travel between any two nodes $i$ and $j$ is denoted with $c_{ij}$. Decision variable $x_{ij}$ is used to describe if the salesperson traverses from node $i$ to node $j$ ($x_{ij}=1$), or not ($x_{ij}=0$). Using this notation, the mathematical model for the TSP is as follows: 

**Objective**
$
\min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij}
\tag{1}
$

**Constraints**

$
\sum_{i=1,\dots,N} x_{ij} = 1,~ \forall~ j = 1,\dots,N 
\tag{2}
$

$
\sum_{j=1,\dots,N} x_{ij} = 1,~ \forall~ i = 1,\dots,N
\tag{3}
$

$
u_{i} - u_{j} + N \cdot x_{ij} \le N -1,~ \forall~ i,j = 1,\dots,N: j \ne 1 ~\text{and}~ i \ne j
\tag{4}
$

$
u_{i} \in \mathbb{Z}^{+},~ \forall~ i = 1,\dots,N
\tag{5}
$

$
x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N
\tag{6}
$

Equation (1) states the objective (distance minimization) by summing up the distances of the selected arcs. Equations (2) and (3)} ensure that the salesperson enters and exists each node exactly once (i.e., that each node is visited exactly once), respectively. Equations (4) restricts solutions with subtour. Finally, Equations (5) and (6) states the domains of the decision variables.

## Python Implementation

### Loading Packages

In [4]:
!pip install numpy
!pip install pandas
!pip install pulp

import numpy as np
import pandas as pd
import pulp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Input Data

In [8]:
input_array = np.array(
    [[1, 2, 3], 
     [4, 5, 6], 
     [7, 8, 9]]
)

df = pd.DataFrame(input_array, columns=['location', 'longitude', 'latitude'])
print(df)

   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
